In [1]:
import pandas as pd
import glob
import unicodedata
import re
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords



In [2]:
!py -m spacy download pt_core_news_sm


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ------------- -------------------------- 4.5/13.0 MB 24.4 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/13.0 MB 24.5 MB/s eta 0:00:01
     --------------------------------------- 13.0/13.0 MB 21.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\miche\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [3]:
# Baixar pacotes do NLTK
nltk.download('punkt')      # Para word_tokenize
nltk.download('stopwords')  # Para stopwords

spacy.cli.download("pt_core_news_sm")
#  Baixar e carregar o modelo de linguagem do Spacy para português
nlp = spacy.load("pt_core_news_sm")  # Modelo para portuguê

[nltk_data] Downloading package punkt to C:\Users\miche/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miche/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


concatenar itens

In [4]:

# Definir o caminho para os arquivos CSV
file_pattern = '../data/raw/itens/itens-parte*.csv'

# Usar glob para buscar todos os arquivos que correspondem ao padrão
file_paths = glob.glob(file_pattern)

# Ler e concatenar todos os arquivos CSV de uma vez
df = pd.concat((pd.read_csv(file) for file in file_paths), ignore_index=True)

# Exibir as primeiras linhas do DataFrame final
print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                      issued                   modified  \
0  2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1  2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2  2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3  2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4  2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               ti

Remover quebra de linhas

In [5]:
df = pd.concat((pd.read_csv(file, lineterminator='\n') for file in file_paths), ignore_index=True)


Contar o número de linhas totalmente vazias

In [6]:
num_linhas_nulas = df.isnull().all(axis=1).sum()
print(f"Número de linhas completamente nulas: {num_linhas_nulas}")

Número de linhas completamente nulas: 0


In [7]:
print("Valores nulos por coluna:")
print(df.isnull().sum())


Valores nulos por coluna:
page        0
url         0
issued      0
modified    0
title       0
body        0
caption     0
dtype: int64


Converter a coluna 'issue' para formato datetime

In [8]:
if 'issued' in df.columns:
    df['issued'] = pd.to_datetime(df['issued'], errors='coerce')
    print("Coluna 'issued' convertida para datetime e salva no dataset.")
else:
    print("Coluna 'issued' não encontrada no DataFrame.")

Coluna 'issued' convertida para datetime e salva no dataset.


Definir as colunas que serão limpas

In [9]:
# Colunas que serão limpas
columns_to_clean = ["title", "body", "caption"]

# Verificando se as colunas existem no DataFrame
print("Colunas encontradas no DataFrame:")
for column in columns_to_clean:
    if column in df.columns:
        print(f"- {column} está presente")
    else:
        print(f"- {column} NÃO está presente")


Colunas encontradas no DataFrame:
- title está presente
- body está presente
- caption está presente


In [10]:
df.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


In [11]:
# Transformando os textos em minúsculas
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].str.lower()

# Exibindo as primeiras linhas após a transformação
print("Texto transformado para minúsculas:")
print(df.head())


Texto transformado para minúsculas:
                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued                   modified  \
0 2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1 2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2 2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3 2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4 2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                   

Normalizando unicode

In [12]:

# Função para normalizar texto Unicode
def normalize_text(text):
    if isinstance(text, str):
        text = text.strip()  # Remover espaços extras no início e fim
        text = unicodedata.normalize("NFKC", text)  # Normalização Unicode
    return text

# Aplicar a normalização nas colunas desejadas
columns_to_clean = ["title", "body", "caption"]

for col in columns_to_clean:
    if col in df.columns:
        df[col] = df[col].apply(normalize_text)

# Exibir as primeiras linhas após a normalização
print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued                   modified  \
0 2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1 2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2 2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3 2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4 2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               title  \

In [13]:
# Função para remover caracteres de escape como "\r" e "\n"
def remove_escape_chars(text):
    return text.replace("\r", " ").replace("\n", " ") if isinstance(text, str) else text

# Função para remover expressões regulares específicas (como URLs, menções e hashtags)
def remove_regex_patterns(text, patterns):
    if isinstance(text, str):
        for pattern in patterns:
            text = re.sub(pattern, '', text)
    return text

# Função para remover números
def remove_numbers(text):
    return re.sub(r'\d+', '', text) if isinstance(text, str) else text

# Função para remover pontuação (mantendo acentuação)
def remove_punctuation(text):
    return re.sub(r'[^\w\sáàãâäéèêëíìîïóòôõöúùûüç]', '', text) if isinstance(text, str) else text

# Função para remover espaços extras
def remove_extra_spaces(text):
    return ' '.join(text.split()) if isinstance(text, str) else text

# Função para substituir hífens por espaços
def replace_hyphens(text):
    return re.sub(r'(?<=\w)-(?=\w)', ' ', text) if isinstance(text, str) else text

# Função para remover colchetes e seu conteúdo
def remove_brackets(text):
    return re.sub(r'\[.*?\]', '', text) if isinstance(text, str) else text

#  Função para remover underscores
def remove_underscores(text):
    return text.replace("_", "") if isinstance(text, str) else text

# Função para remover emojis usando regex
def remove_emojis(text):
    if isinstance(text, str):
        emoji_pattern = re.compile(
            "[\U0001F600-\U0001F64F"  # Emoticons
            "\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
            "\U0001F680-\U0001F6FF"  # Transportes e símbolos adicionais
            "\U0001F1E0-\U0001F1FF"  # Bandeiras (sub-regiões)
            "\U00002500-\U00002BEF"  # Outros símbolos
            "\U00002700-\U000027BF"  # Dingbats
            "\U0001F900-\U0001F9FF"  # Suplementos de emojis
            "\U0001FA70-\U0001FAFF"  # Suplementos adicionais
            "\U00002600-\U000026FF"  # Diversos símbolos (como ♪)
            "\U0001F000-\U0001F02F"  # Cartas de jogo
            "]+",
            flags=re.UNICODE,
        )
        return emoji_pattern.sub(r"", text)  # Remove emojis
    return text  # Retorna o valor original se não for uma string

# Padrões regex a serem removidos
regex_patterns = [
    r"http\S+",   # URLs
    r"@\w+",      # Menções (@user)
    r"#\w+",      # Hashtags (#palavra)
    r"º",         # Caractere º
]

# Aplicando as funções de pré-processamento nas colunas
columns_to_clean = ["title", "body", "caption"]

for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].astype(str)                   # Converter para string (evita erros)
        df[column] = df[column].apply(remove_escape_chars)    # Remover caracteres como \r e \n
        df[column] = df[column].apply(replace_hyphens)        # Substituir hífens por espaços
        df[column] = df[column].apply(lambda x: remove_regex_patterns(x, regex_patterns))  # Remover padrões regex
        df[column] = df[column].apply(remove_numbers)         # Remover números
        df[column] = df[column].apply(remove_punctuation)     # Remover pontuação
        df[column] = df[column].apply(remove_brackets)        # Remover colchetes e conteúdo dentro deles
        df[column] = df[column].apply(remove_underscores)     # Remover underscores
        df[column] = df[column].apply(remove_emojis)          # Remover emojis
        df[column] = df[column].apply(remove_extra_spaces)    # Remover espaços extras

# Exibir as primeiras linhas após o pré-processamento
print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued                   modified  \
0 2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1 2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2 2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3 2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4 2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               title  \

tokenização e remoção de stopwords

In [14]:
# Função para tokenizar o texto
def tokenize_text(text):
    return word_tokenize(text) if isinstance(text, str) else []

#  Função para remover stopwords de uma lista de tokens
def remove_stopwords(tokens, language='portuguese'):
    stop_words = set(stopwords.words(language))  # Definir as stopwords para o idioma desejado
    return [word for word in tokens if word.lower() not in stop_words]  # Remove stopwords e converte para minúsculas

# Aplicando tokenização e remoção de stopwords no DataFrame
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(lambda x: ' '.join(remove_stopwords(tokenize_text(str(x)))))

# Exibir as primeiras linhas do DataFrame após a limpeza
print(df.head())

                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued                   modified  \
0 2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1 2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2 2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3 2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4 2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               title  \

In [15]:
# Função para lematizar texto com POS usando Spacy
def lemmatize_text(text):
    if isinstance(text, str):  # Garantir que a entrada seja uma string
        doc = nlp(text)  # Processar o texto com Spacy
        lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop]  # Lematizar e remover stopwords
        return ' '.join(lemmatized_tokens)  # Retornar texto limpo
    return text  # Retornar valor original caso não seja string

# Aplicando a função de lematização no DataFrame
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(lemmatize_text)

# Exibir as primeiras linhas do DataFrame após o pré-processamento
print(df.head())

                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued                   modified  \
0 2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1 2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2 2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3 2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4 2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               title  \

In [18]:
# Salvar o DataFrame processado em um arquivo CSV
df.to_csv('../data/preprocessed/itens/dados_preprocessados.csv', index=False)

print("Arquivo CSV salvo com sucesso!")


Arquivo CSV salvo com sucesso!
